In [3]:
!pip install ipywidgets --quiet
import ipywidgets as widgets
from IPython.display import display, clear_output


In [4]:
import os
import shutil
import hashlib
import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

VCS_DIR = ".vcs"
COMMITS_DIR = os.path.join(VCS_DIR, "commits")
LOG_FILE = os.path.join(VCS_DIR, "log.txt")

def init_vcs():
    if not os.path.exists(VCS_DIR):
        os.makedirs(COMMITS_DIR)
        with open(LOG_FILE, "w") as f:
            f.write("Mini VCS initialized\n")
        out.value = "✅ Initialized Mini VCS."
    else:
        out.value = "⚠️ VCS already initialized."

def get_commit_hash():
    now = datetime.datetime.now().isoformat()
    return hashlib.sha1(now.encode()).hexdigest()[:6]

def commit_file(msg):
    if not os.path.exists(VCS_DIR):
        out.value = "⚠️ Please initialize VCS first."
        return

    commit_hash = get_commit_hash()
    commit_path = os.path.join(COMMITS_DIR, commit_hash)
    os.makedirs(commit_path)

    for file in os.listdir():
        if file.startswith(".") or file.endswith(".ipynb"):
            continue
        if os.path.isfile(file):
            shutil.copy(file, commit_path)

    with open(LOG_FILE, "a") as f:
        f.write(f"{commit_hash} - {msg}\n")

    out.value = f"✅ Committed as: {commit_hash}"

def view_log():
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, "r") as f:
            log_output.value = f.read()
    else:
        log_output.value = "⚠️ No log found."

def do_checkout(hash_code):
    path = os.path.join(COMMITS_DIR, hash_code)
    if not os.path.exists(path):
        out.value = "❌ Commit hash not found."
        return

    for file in os.listdir():
        if file.startswith(".") or file.endswith(".ipynb"):
            continue
        if os.path.isfile(file):
            os.remove(file)

    for file in os.listdir(path):
        shutil.copy(os.path.join(path, file), file)

    out.value = f"✅ Checked out: {hash_code}"

# Widgets
init_btn = widgets.Button(description="Init VCS")
commit_btn = widgets.Button(description="Commit")
log_btn = widgets.Button(description="View Log")
checkout_btn = widgets.Button(description="Checkout")

msg_input = widgets.Text(description="Message")
hash_input = widgets.Text(description="Commit Hash")
out = widgets.Label()
log_output = widgets.Textarea(layout={"height": "200px", "width": "100%"})

# Actions
init_btn.on_click(lambda b: init_vcs())
commit_btn.on_click(lambda b: commit_file(msg_input.value))
log_btn.on_click(lambda b: view_log())
checkout_btn.on_click(lambda b: do_checkout(hash_input.value))

# Display
display(widgets.VBox([
    widgets.HTML("<h3>🗃️ Mini VCS in Colab</h3>"),
    widgets.HBox([init_btn]),
    widgets.HBox([msg_input, commit_btn]),
    widgets.HBox([hash_input, checkout_btn]),
    log_btn,
    log_output,
    out
]))
